**Data loader**

In [63]:
import os
import pandas as pd
from urllib import urlopen
import os

decklists_path = 'data/decks/'
matches_path = 'data/matches/'

class Loader():
    def __init__(self):
        self.deckfiles = []
        self.decks = []
        self.cards = []
        self.player_archetype = []
        self.allcards = []
        
    def loadAllDeckFilenames(self):
        if len(self.deckfiles) == 0:
            tmp = os.listdir(decklists_path)
            deckfiles = []
            for i in tmp:
                deckfiles.append(i.replace('~', ''))
            self.deckfiles = list(set(deckfiles))
        
    def loadDecks(self, filename):
        data = pd.read_json('data/decks/'+filename)
        return data['cards']
    
    def loadPlayerArchetype(self, filename):
        data = pd.read_json('data/decks/'+filename)
        tmp = []
        for i in range(0, len(data)):
            tmp.append({'draft': data['draft'][i], 'player': data['player'][i], 'archetype': data['name'][i]})
        return tmp
            
    def loadAllPlayerArchetype(self):
        self.loadAllDeckFilenames()
        for i in self.deckfiles:
            tmp = self.loadPlayerArchetype(i)
            for j in tmp:
                self.player_archetype.append(j)
    
    def loadAllDecks(self):
        self.loadAllDeckFilenames()
        for i in self.deckfiles:
            tmp_decks = self.loadDecks(i)
            for d in tmp_decks:
                self.decks.append(d)
    
    def loadAllUsedCards(self):
        if len(self.cards) == 0:
            self.loadAllDecks()
            for i in self.decks:
                for j in i:
                    if "(ORI)" not in j:
                        self.cards.append(j)
            self.cards = sorted(list(set(self.cards)))
    
    def downloadAllCardsData(self):
        URL = 'https://mtgjson.com/json/AllCards.json.zip'
        url = urlopen(URL)
        output = open('zippedCards.zip', 'wb')
        output.write(url.read())
        output.close()
        os.system('unzip zippedCards.zip')
        os.remove('zippedCards.zip')
        self.allcards = pd.read_json('AllCards.json')
    
    def loadAllCardsData(self):
        if os.path.isfile('AllCards.json') == False:
            self.downloadAllCardsData()
        if len(self.allcards) == 0:
            self.allcards = pd.read_json('AllCards.json')
            
    def loadCurrentCubeCards(self, filename):
        self.cube = []
        my_file = open(filename, 'r')
        my_string = my_file.read()
        my_list = my_string.split('\r\n')
        for i in my_list:
            tmp = i.split(' ')
            if len(tmp) > 1:
                cardname = " ".join(tmp[1:])
                self.cube.append(cardname)
            
    def findInvalidCardnames(self, cubefile):
        self.invalid_names = []
        self.loadAllCardsData()
        self.loadAllUsedCards()
        self.loadCurrentCubeCards(cubefile)
        for i in self.cards:
            if i not in self.allcards:
                self.invalid_names.append(i)
        for i in self.cube:
            if i not in self.allcards:
                self.invalid_names.append(i)
        
        print self.invalid_names
                
    def correctNamesInAllDecks(self):
        self.loadAllDeckFilenames()
        self.findInvalidCardnames()
        for deckfile in self.deckfiles:
            my_file = open('data/decks/'+deckfile, 'r')
            tmp = my_file.read()
            for j in self.invalid_names:
                tmp = tmp.replace(j, cards_to_correct[j])
            my_file.close()
            out_file = open('data/decks/'+deckfile, 'w')
            out_file.write(tmp)
            out_file.close()
        
loader = Loader()

loader.loadAllUsedCards()

cubefile = 'numbered_18_3.txt'

loader.findInvalidCardnames(cubefile)


        

[]


**Data Saver**